### Full d20 pretrain

I'm now going to try the full d20 pretrain on an 8xH100 machine on lambda cloud.

First I'll follow the instructions in `trying-lambda-cloud.ipynb` to get ready.

If I'm able to get a machine in the same region as my test (us-south-2) then I can use the same storage and the data files and tokenizer should already be there. If not, I'll download the data files and scp the tokenizer from my mac:

```
cd tokenizer-from-challenge-25
scp my-tokenizer.pkl ubuntu@[ip]:/home/ubuntu/mynanochat/
scp token_bytes.pt ubuntu@[ip]:/home/ubuntu/mynanochat/
```

I was able to use the same region. It says the machine could take up to 20 minutes to boot. Took < 5.

```
ubuntu@192-222-53-36:~$ nvidia-smi 
Sun Nov 16 16:08:09 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.148.08             Driver Version: 570.148.08     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA H100 80GB HBM3          On  |   00000000:61:00.0 Off |                    0 |
| N/A   26C    P0             70W /  700W |       0MiB /  81559MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+------------------------+----------------------+
|   1  NVIDIA H100 80GB HBM3          On  |   00000000:62:00.0 Off |                    0 |
| N/A   27C    P0             74W /  700W |       0MiB /  81559MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+------------------------+----------------------+
|   2  NVIDIA H100 80GB HBM3          On  |   00000000:63:00.0 Off |                    0 |
| N/A   29C    P0             71W /  700W |       0MiB /  81559MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+------------------------+----------------------+
|   3  NVIDIA H100 80GB HBM3          On  |   00000000:64:00.0 Off |                    0 |
| N/A   26C    P0             76W /  700W |       0MiB /  81559MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+------------------------+----------------------+
|   4  NVIDIA H100 80GB HBM3          On  |   00000000:6A:00.0 Off |                    0 |
| N/A   25C    P0            N/A  /  700W |       0MiB /  81559MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+------------------------+----------------------+
|   5  NVIDIA H100 80GB HBM3          On  |   00000000:6B:00.0 Off |                    0 |
| N/A   28C    P0             72W /  700W |       0MiB /  81559MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+------------------------+----------------------+
|   6  NVIDIA H100 80GB HBM3          On  |   00000000:6C:00.0 Off |                    0 |
| N/A   27C    P0             68W /  700W |       0MiB /  81559MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+------------------------+----------------------+
|   7  NVIDIA H100 80GB HBM3          On  |   00000000:6D:00.0 Off |                    0 |
| N/A   25C    P0             69W /  700W |       0MiB /  81559MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+------------------------+----------------------+
                                                                                         
+-----------------------------------------------------------------------------------------+
| Processes:                                                                              |
|  GPU   GI   CI              PID   Type   Process name                        GPU Memory |
|        ID   ID                                                               Usage      |
|=========================================================================================|
|  No running processes found                                                             |
+-----------------------------------------------------------------------------------------+
```

In tmux shell:

```
source .venv/bin/activate

cd challenge-25-pretrain-d20

export PYTHONPATH=../my_nanochat/

torchrun --standalone --nproc_per_node=8 -m scripts.my_base_train -- --depth=20 --run=challenge-25-4 > train_output_004.txt 2>&1
```

I started with --proc_per_nod=1 by mistake. (That's what I called run=challenge-25-3.) Killed as soon as I noticed tok/sec was similar to before and started as above.

Now seeing over 1M tokens per second.

```
step 00026/21400 (0.12%) | loss: 6.478619 | grad norm: 1.4101 | lrm: 1.00 | dt: 486.55ms | tok/sec: 1,077,563 | mfu: -1.00 | total time: 0.13m
step 00027/21400 (0.13%) | loss: 6.464063 | grad norm: 1.0951 | lrm: 1.00 | dt: 489.48ms | tok/sec: 1,071,108 | mfu: -1.00 | total time: 0.14m
```

```
Number of parameters: 560,988,160
Calculated num iterations from target_param_data_ratio as 21,400
```

Here's the firsrt sampling at step 2000. Definitely better than anything before.

```
<|bos|>The capital of France is Paris. The capital of France is Paris. The
<|bos|>The chemical symbol of gold is 1. It is a soft metal, and
<|bos|>If yesterday was Friday, then tomorrow will be Thursday, and if tomorrow is Wednesday, then tomorrow
<|bos|>The opposite of hot is cold. The opposite of hot is hot. The
<|bos|>The planets of the solar system are: Mercury, Venus, Earth, Mars, Jupiter,
<|bos|>My favorite color is red. I think it’s a good color for
<|bos|>If 5*x + 3 = 13, then x is 13. If 5*x + 
```

Here's the next at step 4000.

```
<|bos|>The capital of France is Paris, the capital of France. It is the
<|bos|>The chemical symbol of gold is Au. The symbol of silver is Au. The
<|bos|>If yesterday was Friday, then tomorrow will be tomorrow. If you are a student, then you
<|bos|>The opposite of hot is cold. The opposite of hot is cold. The
<|bos|>The planets of the solar system are: Jupiter, Saturn, Uranus, Neptune, and Pluto
<|bos|>My favorite color is red. I love it because it is the color
<|bos|>If 5*x + 3 = 13, then x is 5. If 5*x + 
```

at 6000:

```
<|bos|>The capital of France is Paris, the capital of the United States is Washington
<|bos|>The chemical symbol of gold is 24. Gold is a soft, malleable metal
<|bos|>If yesterday was Friday, then tomorrow will be Monday. If you are a parent, you will
<|bos|>The opposite of hot is cold. The opposite of hot is cold. The
<|bos|>The planets of the solar system are: Mercury, Venus, Earth, Mars, Jupiter,
<|bos|>My favorite color is red. It is the color of the sky,
<|bos|>If 5*x + 3 = 13, then x is a square. If 5*x + 
```

at 8000:

```
<|bos|>The capital of France is Paris, the capital of France is Paris, the
<|bos|>The chemical symbol of gold is Au, and the chemical symbol of silver is Ag
<|bos|>If yesterday was Friday, then tomorrow will be Sunday. If tomorrow is Sunday, then tomorrow will
<|bos|>The opposite of hot is cold. The opposite of hot is cold. The
<|bos|>The planets of the solar system are: Mercury, Venus, Earth, Mars, Jupiter,
<|bos|>My favorite color is red, and I love the way it is.
<|bos|>If 5*x + 3 = 13, then x is the square root of 5. If 5
```

at 10,000:

```
<|bos|>The capital of France is Paris, and the city is the most populous in
<|bos|>The chemical symbol of gold is Au. The symbol of gold is Au is a
<|bos|>If yesterday was Friday, then tomorrow will be Friday. If today is Friday, then tomorrow will
<|bos|>The opposite of hot is cold. The opposite of hot is cold. The
<|bos|>The planets of the solar system are: Mercury, Venus, Earth, Mars, Jupiter,
<|bos|>My favorite color is red. It is the color of love, of
<|bos|>If 5*x + 3 = 13, then x is 5*x + 3 = 13
```

at 12,000:

```
<|bos|>The capital of France is Paris. The capital of France is Paris. The
<|bos|>The chemical symbol of gold is Au. The name of the metal is derived from
<|bos|>If yesterday was Friday, then tomorrow will be Friday. This is a good way to keep track
<|bos|>The opposite of hot is cold. The opposite of cold is hot. The
<|bos|>The planets of the solar system are: Earth, Mars, Jupiter, Saturn, Uranus,
<|bos|>My favorite color is red. I love red. I love red.
<|bos|>If 5*x + 3 = 13, then x is 3 times 13. So, 5
```

At 14,000:

```
<|bos|>The capital of France is Paris, and the capital of France is Paris.
<|bos|>The chemical symbol of gold is Au. The atomic number of gold is 79
<|bos|>If yesterday was Friday, then tomorrow will be Monday. If yesterday was Monday, then tomorrow will
<|bos|>The opposite of hot is cold. The opposite of cold is hot. The
<|bos|>The planets of the solar system are: Mercury, Venus, Earth, Mars, Jupiter,
<|bos|>My favorite color is red. It is the color of the blood of
<|bos|>If 5*x + 3 = 13, then x is a multiple of 3. The answer is 
```

(The atomic number of gold is 79!)

At 16,000:

```
<|bos|>The capital of France is Paris, and the capital of France is Paris.
<|bos|>The chemical symbol of gold is Au. The chemical symbol of gold is Au.
<|bos|>If yesterday was Friday, then tomorrow will be Monday. If tomorrow is Tuesday, then tomorrow will
<|bos|>The opposite of hot is cold. The opposite of hot is cold. The
<|bos|>The planets of the solar system are: Mercury, Venus, Earth, Mars, Jupiter,
<|bos|>My favorite color is red. It’s the color of the blood of
<|bos|>If 5*x + 3 = 13, then x is 13. If 5*x + 
```

It's pretty crazy that some of the early people to see results like this thought if they keep scaling up the model will be able to do advanced "thinking." These samples scream "I'm getting pretty good at completing prompts" not "I'm understanding" and this was before we realized just how much "thinking" or "understanding" is about completing prompts.

At 18,000:

```
<|bos|>The capital of France is Paris. It is the capital of France. It
<|bos|>The chemical symbol of gold is Au. The chemical symbol of silver is Ag.
<|bos|>If yesterday was Friday, then tomorrow will be Monday. If tomorrow is Monday, then tomorrow will
<|bos|>The opposite of hot is cold. The opposite of hot is cold. The
<|bos|>The planets of the solar system are: Mercury, Venus, Earth, Mars, Jupiter,
<|bos|>My favorite color is red. It is the color of the blood of
<|bos|>If 5*x + 3 = 13, then x is 13. If 5*x + 
```

At 20,000:

```
<|bos|>The capital of France is Paris. It is the second largest city in France
<|bos|>The chemical symbol of gold is Au. It is a soft, malleable, ductile
<|bos|>If yesterday was Friday, then tomorrow will be Monday, and so on. The only way to
<|bos|>The opposite of hot is cold. The opposite of cold is hot. The
<|bos|>The planets of the solar system are: Mercury, Venus, Earth, Mars, Jupiter,
<|bos|>My favorite color is red. It’s the color of the blood of
<|bos|>If 5*x + 3 = 13, then x is a factor of 2. If 5*
```

Paris is not the second largest city.

At 21,400:

```
<|bos|>The capital of France is Paris. It is the largest city in France and
<|bos|>The chemical symbol of gold is Au. It is a soft, malleable, ductile
<|bos|>If yesterday was Friday, then tomorrow will be Monday. If tomorrow is Monday, then tomorrow will
<|bos|>The opposite of hot is cold. The opposite of cold is hot. The
<|bos|>The planets of the solar system are: Mercury, Venus, Earth, Mars, Jupiter,
<|bos|>My favorite color is red. It is the color of the blood of
<|bos|>If 5*x + 3 = 13, then x is a factor of 5. If 5*
```

Last 100 lines of output from job:
```
step 21374/21400 (99.88%) | loss: 2.741960 | grad norm: 0.1850 | lrm: 0.01 | dt: 480.02ms | tok/sec: 1,092,213 | mfu: -1.00 | total time: 171.83m
step 21375/21400 (99.88%) | loss: 2.742765 | grad norm: 0.1700 | lrm: 0.01 | dt: 479.68ms | tok/sec: 1,093,004 | mfu: -1.00 | total time: 171.84m
step 21376/21400 (99.89%) | loss: 2.749703 | grad norm: 0.1950 | lrm: 0.01 | dt: 479.89ms | tok/sec: 1,092,508 | mfu: -1.00 | total time: 171.85m
step 21377/21400 (99.89%) | loss: 2.750738 | grad norm: 0.1698 | lrm: 0.01 | dt: 479.51ms | tok/sec: 1,093,382 | mfu: -1.00 | total time: 171.86m
step 21378/21400 (99.90%) | loss: 2.750943 | grad norm: 0.1935 | lrm: 0.01 | dt: 478.80ms | tok/sec: 1,095,013 | mfu: -1.00 | total time: 171.86m
step 21379/21400 (99.90%) | loss: 2.752251 | grad norm: 0.1763 | lrm: 0.00 | dt: 480.70ms | tok/sec: 1,090,665 | mfu: -1.00 | total time: 171.87m
step 21380/21400 (99.91%) | loss: 2.756554 | grad norm: 0.1785 | lrm: 0.00 | dt: 479.59ms | tok/sec: 1,093,201 | mfu: -1.00 | total time: 171.88m
step 21381/21400 (99.91%) | loss: 2.748836 | grad norm: 0.1720 | lrm: 0.00 | dt: 479.94ms | tok/sec: 1,092,413 | mfu: -1.00 | total time: 171.89m
step 21382/21400 (99.92%) | loss: 2.753904 | grad norm: 0.1738 | lrm: 0.00 | dt: 480.36ms | tok/sec: 1,091,445 | mfu: -1.00 | total time: 171.90m
step 21383/21400 (99.92%) | loss: 2.753519 | grad norm: 0.1712 | lrm: 0.00 | dt: 478.78ms | tok/sec: 1,095,056 | mfu: -1.00 | total time: 171.90m
step 21384/21400 (99.93%) | loss: 2.751234 | grad norm: 0.1663 | lrm: 0.00 | dt: 480.38ms | tok/sec: 1,091,394 | mfu: -1.00 | total time: 171.91m
step 21385/21400 (99.93%) | loss: 2.743604 | grad norm: 0.1646 | lrm: 0.00 | dt: 479.27ms | tok/sec: 1,093,924 | mfu: -1.00 | total time: 171.92m
step 21386/21400 (99.93%) | loss: 2.732429 | grad norm: 0.1636 | lrm: 0.00 | dt: 478.18ms | tok/sec: 1,096,426 | mfu: -1.00 | total time: 171.93m
step 21387/21400 (99.94%) | loss: 2.741124 | grad norm: 0.1752 | lrm: 0.00 | dt: 480.91ms | tok/sec: 1,090,190 | mfu: -1.00 | total time: 171.94m
step 21388/21400 (99.94%) | loss: 2.735194 | grad norm: 0.1660 | lrm: 0.00 | dt: 478.97ms | tok/sec: 1,094,612 | mfu: -1.00 | total time: 171.94m
step 21389/21400 (99.95%) | loss: 2.731359 | grad norm: 0.1669 | lrm: 0.00 | dt: 480.26ms | tok/sec: 1,091,675 | mfu: -1.00 | total time: 171.95m
step 21390/21400 (99.95%) | loss: 2.742557 | grad norm: 0.1905 | lrm: 0.00 | dt: 478.04ms | tok/sec: 1,096,737 | mfu: -1.00 | total time: 171.96m
step 21391/21400 (99.96%) | loss: 2.740270 | grad norm: 0.1763 | lrm: 0.00 | dt: 481.01ms | tok/sec: 1,089,977 | mfu: -1.00 | total time: 171.97m
step 21392/21400 (99.96%) | loss: 2.704566 | grad norm: 0.1630 | lrm: 0.00 | dt: 480.92ms | tok/sec: 1,090,174 | mfu: -1.00 | total time: 171.98m
step 21393/21400 (99.97%) | loss: 2.707926 | grad norm: 0.1676 | lrm: 0.00 | dt: 479.31ms | tok/sec: 1,093,829 | mfu: -1.00 | total time: 171.98m
step 21394/21400 (99.97%) | loss: 2.718694 | grad norm: 0.1748 | lrm: 0.00 | dt: 479.41ms | tok/sec: 1,093,608 | mfu: -1.00 | total time: 171.99m
step 21395/21400 (99.98%) | loss: 2.728112 | grad norm: 0.1696 | lrm: 0.00 | dt: 477.57ms | tok/sec: 1,097,815 | mfu: -1.00 | total time: 172.00m
step 21396/21400 (99.98%) | loss: 2.722741 | grad norm: 0.1687 | lrm: 0.00 | dt: 480.46ms | tok/sec: 1,091,218 | mfu: -1.00 | total time: 172.01m
step 21397/21400 (99.99%) | loss: 2.729883 | grad norm: 0.1745 | lrm: 0.00 | dt: 482.27ms | tok/sec: 1,087,122 | mfu: -1.00 | total time: 172.02m
step 21398/21400 (99.99%) | loss: 2.733281 | grad norm: 0.1718 | lrm: 0.00 | dt: 481.44ms | tok/sec: 1,088,994 | mfu: -1.00 | total time: 172.02m
step 21399/21400 (100.00%) | loss: 2.746857 | grad norm: 0.1814 | lrm: 0.00 | dt: 477.22ms | tok/sec: 1,098,640 | mfu: -1.00 | total time: 172.03m
step 21400 | Validation bpb: 0.8135
Evaluating: hellaswag_zeroshot (0-shot, type: multiple_choice)... accuracy: 0.4320 | centered: 0.2427 | time: 0.72s
Evaluating: jeopardy (10-shot, type: language_modeling)... accuracy: 0.1320 | centered: 0.1320 | time: 0.70s
Evaluating: bigbench_qa_wikidata (10-shot, type: language_modeling)... accuracy: 0.5500 | centered: 0.5500 | time: 0.71s
Evaluating: arc_easy (10-shot, type: multiple_choice)... accuracy: 0.6640 | centered: 0.5520 | time: 0.78s
Evaluating: arc_challenge (10-shot, type: multiple_choice)... accuracy: 0.3840 | centered: 0.1787 | time: 0.83s
Evaluating: copa (0-shot, type: multiple_choice)... accuracy: 0.6900 | centered: 0.3800 | time: 0.14s
Evaluating: commonsense_qa (10-shot, type: multiple_choice)... accuracy: 0.2160 | centered: 0.0200 | time: 0.86s
Evaluating: piqa (10-shot, type: multiple_choice)... accuracy: 0.6920 | centered: 0.3840 | time: 0.71s
Evaluating: openbook_qa (0-shot, type: multiple_choice)... accuracy: 0.3600 | centered: 0.1467 | time: 0.62s
Evaluating: lambada_openai (0-shot, type: language_modeling)... accuracy: 0.3960 | centered: 0.3960 | time: 0.65s
Evaluating: hellaswag (10-shot, type: multiple_choice)... accuracy: 0.4520 | centered: 0.2693 | time: 1.38s
Evaluating: winograd (0-shot, type: schema)... accuracy: 0.6410 | centered: 0.2821 | time: 0.33s
Evaluating: winogrande (0-shot, type: schema)... accuracy: 0.5220 | centered: 0.0440 | time: 0.62s
Evaluating: bigbench_dyck_languages (10-shot, type: language_modeling)... accuracy: 0.0980 | centered: 0.0980 | time: 0.70s
Evaluating: agi_eval_lsat_ar (3-shot, type: multiple_choice)... accuracy: 0.2217 | centered: 0.0272 | time: 0.59s
Evaluating: bigbench_cs_algorithms (10-shot, type: language_modeling)... accuracy: 0.3580 | centered: 0.3580 | time: 0.68s
Evaluating: bigbench_operators (10-shot, type: language_modeling)... accuracy: 0.1667 | centered: 0.1667 | time: 0.29s
Evaluating: bigbench_repeat_copy_logic (10-shot, type: language_modeling)... accuracy: 0.0000 | centered: 0.0000 | time: 0.04s
Evaluating: squad (10-shot, type: language_modeling)... accuracy: 0.2360 | centered: 0.2360 | time: 1.07s
Evaluating: coqa (0-shot, type: language_modeling)... accuracy: 0.1800 | centered: 0.1800 | time: 0.75s
Evaluating: boolq (10-shot, type: multiple_choice)... accuracy: 0.5240 | centered: -0.2526 | time: 1.29s
Evaluating: bigbench_language_identification (10-shot, type: multiple_choice)... accuracy: 0.2680 | centered: 0.1947 | time: 2.12s
Step 21400: CORE metric: 0.2084
<|bos|>The capital of France is Paris. It is the largest city in France and
<|bos|>The chemical symbol of gold is Au. It is a soft, malleable, ductile
<|bos|>If yesterday was Friday, then tomorrow will be Monday. If tomorrow is Monday, then tomorrow will
<|bos|>The opposite of hot is cold. The opposite of cold is hot. The
<|bos|>The planets of the solar system are: Mercury, Venus, Earth, Mars, Jupiter,
<|bos|>My favorite color is red. It is the color of the blood of
<|bos|>If 5*x + 3 = 13, then x is a factor of 5. If 5*
saved model to /home/ubuntu/mynanochat/base_checkpoints/d20/model_021400.pt
saved metadata to /home/ubuntu/mynanochat/base_checkpoints/d20/meta_021400.json
[W1116 19:16:47.976035058 AllocatorConfig.cpp:28] Warning: PYTORCH_CUDA_ALLOC_CONF is deprecated, use PYTORCH_ALLOC_CONF instead (function operator())
[W1116 19:16:47.151341352 AllocatorConfig.cpp:28] Warning: PYTORCH_CUDA_ALLOC_CONF is deprecated, use PYTORCH_ALLOC_CONF instead (function operator())
[W1116 19:16:47.370174970 AllocatorConfig.cpp:28] Warning: PYTORCH_CUDA_ALLOC_CONF is deprecated, use PYTORCH_ALLOC_CONF instead (function operator())
[W1116 19:16:47.392311868 AllocatorConfig.cpp:28] Warning: PYTORCH_CUDA_ALLOC_CONF is deprecated, use PYTORCH_ALLOC_CONF instead (function operator())
[W1116 19:16:47.524902948 AllocatorConfig.cpp:28] Warning: PYTORCH_CUDA_ALLOC_CONF is deprecated, use PYTORCH_ALLOC_CONF instead (function operator())
saved optimizer to /home/ubuntu/mynanochat/base_checkpoints/d20/optim_021400_rank0.pt
Peak memory usage: 75422.02MiB
Total training time: 172.03m
Minimum validation bpb: 0.8135
wandb: updating run metadata
wandb: uploading history steps 311-311, summary, console lines 21838-21859
wandb: 
wandb: Run history:
wandb:         core_metric ▁▄▄▆▆▇▅▇▇██
wandb:                step ▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇█████
wandb: total_training_time ▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇███
wandb:            train/dt ▇██▆▇▇▆▆▇▇▆▅▇▆▅▇▇▇█▇▇▇▇▁▇▇▇▂▆▅▇▆▆▇▇▇█▂▆▇
wandb:     train/grad_norm ▂▂▂▃▂▂▂▁▂▁▁▁▁▁▁▁▂▂▂▁▂▂▇▂▂▂▅▂█▃▄▁▆▁▁▃▂▂▂▁
wandb:          train/loss █▆▄▅▄▃▃▃▃▃▃▂▃▂▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁
wandb:           train/lrm ███████████████████████████████▇▆▆▅▄▄▄▂▁
wandb:   train/tok_per_sec ▁▇█████████▇████████████████████████████
wandb:             val/bpb █▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb: 
wandb: Run summary:
wandb:         core_metric 0.20842
wandb:                step 21400
wandb: total_training_time 10321.90252
wandb:            train/dt 0.48043
wandb:     train/grad_norm 0.17181
wandb:          train/loss 2.71189
wandb:           train/lrm 0.02336
wandb:   train/tok_per_sec 1091299
wandb:             val/bpb 0.81348
wandb: 
wandb: 🚀 View run challenge-25-4 at: https://wandb.ai/ericsilberstein-self/my-nanochat/runs/i5o3evvu
wandb: ⭐️ View project at: https://wandb.ai/ericsilberstein-self/my-nanochat
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20251116_161520-i5o3evvu/logs
[W1116 19:16:53.795706450 AllocatorConfig.cpp:28] Warning: PYTORCH_CUDA_ALLOC_CONF is deprecated, use PYTORCH_ALLOC_CONF instead (function operator())
[W1116 19:16:53.310833985 AllocatorConfig.cpp:28] Warning: PYTORCH_CUDA_ALLOC_CONF is deprecated, use PYTORCH_ALLOC_CONF instead (function operator())
[W1116 19:16:55.520616915 AllocatorConfig.cpp:28] Warning: PYTORCH_CUDA_ALLOC_CONF is deprecated, use PYTORCH_ALLOC_CONF instead (function operator())
```

### Copying trained model to my laptop

I want to start getting more organized about where I keep things.

On my laptop, at root of this repo:

```
mkdir files_copied_from_servers
cd files_copied_from_servers
mkdir challenge-25-pretrain-d20
cd challenge-25-pretrain-d20
mv ~/Documents/tokenizer-from-challenge-25/* .
rmdir ~/Documents/tokenizer-from-challenge-25
ls -lh
total 2232
-rw-r--r--  1 ericsilberstein  staff   826K Nov 16 08:52 my-tokenizer.pkl
-rw-r--r--  1 ericsilberstein  staff   258K Nov 16 08:52 token_bytes.pt
-rw-r--r--  1 ericsilberstein  staff    25K Nov 16 09:00 train_output_001.txt
```

Added to `.gitignore`

When the model is done training I'll scp it to there:

```
scp -r ubuntu@[ip]:/home/ubuntu/mynanochat/base_checkpoints .
scp -r ubuntu@[ip]:/home/ubuntu/nanogpt-learning/challenge-25-pretrain-d20/train_output_004.txt .

ls -lh base_checkpoints/d20 
total 4884856
-rw-r--r--  1 ericsilberstein  staff   847B Nov 16 14:18 meta_021400.json
-rw-r--r--  1 ericsilberstein  staff   1.9G Nov 16 14:20 model_021400.pt
-rw-r--r--  1 ericsilberstein  staff   389M Nov 16 14:21 optim_021400_rank0.pt
```

Now terminate machine.

That was 3.31 hours @ \\$23.92 / hr = \\$79.24